In [1]:
import cvxpy as cvx
import numpy as np
import timeit
import sys
sys.path.append('../')
from ios import *

In [2]:
bigram = get_bigram('../data/processed/bigram.npy')
plainseq = get_plaintext('../data/408plaincleaned')
plainseq = tointseq(plainseq)
plainmat = np.reshape(plainseq, (24, 17))

def toCharMat(intMat, tour=None):
    res = np.chararray(intMat.shape, unicode=True)
    if tour is None:
        for i in range(intMat.shape[1]):
            res[:,i] = tocharseq(intMat[:,i])
    else:
        for i,j in enumerate(tour):
            res[:,i] = tocharseq(intMat[:,j])
    return res   

charmat = toCharMat(plainmat)
print(charmat)

[['I' 'L' 'I' 'K' 'E' 'K' 'I' 'L' 'L' 'I' 'N' 'G' 'P' 'E' 'O' 'P' 'L']
 ['E' 'B' 'E' 'C' 'A' 'U' 'S' 'E' 'I' 'T' 'I' 'S' 'S' 'O' 'M' 'U' 'C']
 ['H' 'F' 'U' 'N' 'I' 'T' 'I' 'S' 'M' 'O' 'R' 'E' 'F' 'U' 'N' 'T' 'H']
 ['A' 'N' 'K' 'I' 'L' 'L' 'I' 'N' 'G' 'W' 'I' 'L' 'D' 'G' 'A' 'M' 'E']
 ['I' 'N' 'T' 'H' 'E' 'F' 'O' 'R' 'R' 'E' 'S' 'T' 'B' 'E' 'C' 'A' 'U']
 ['S' 'E' 'M' 'A' 'N' 'I' 'S' 'T' 'H' 'E' 'M' 'O' 'S' 'T' 'D' 'A' 'N']
 ['G' 'E' 'R' 'O' 'U' 'S' 'A' 'N' 'I' 'M' 'A' 'L' 'O' 'F' 'A' 'L' 'L']
 ['T' 'O' 'K' 'I' 'L' 'L' 'S' 'O' 'M' 'E' 'T' 'H' 'I' 'N' 'G' 'G' 'I']
 ['V' 'E' 'S' 'M' 'E' 'T' 'H' 'E' 'M' 'O' 'S' 'T' 'T' 'H' 'R' 'I' 'L']
 ['L' 'I' 'N' 'G' 'E' 'X' 'P' 'E' 'R' 'E' 'N' 'C' 'E' 'I' 'T' 'I' 'S']
 ['E' 'V' 'E' 'N' 'B' 'E' 'T' 'T' 'E' 'R' 'T' 'H' 'A' 'N' 'G' 'E' 'T']
 ['T' 'I' 'N' 'G' 'Y' 'O' 'U' 'R' 'R' 'O' 'C' 'K' 'S' 'O' 'F' 'F' 'W']
 ['I' 'T' 'H' 'A' 'G' 'I' 'R' 'L' 'T' 'H' 'E' 'B' 'E' 'S' 'T' 'P' 'A']
 ['R' 'T' 'O' 'F' 'I' 'T' 'I' 'S' 'T' 'H' 'A' 'T' 'W' 'H' 'E' 'N' 'I']
 ['D' 

In [3]:
# column permutation and revsersible mapping

perm = np.random.permutation(17)
perm_plainmat = plainmat[:,perm]
# print(toCharMat(perm_plainmat, perm=perm))

mapback = dict()
for i,j in enumerate(perm):
    mapback[i] = j

In [4]:
def col2col(a,b):
    logp = 0
    for i,j in zip(a,b):
        logp += np.log(bigram[i,j])
    return logp

def colscore(mat):
    score = 0
    for i in range(mat.shape[1] - 1):
        score += col2col(mat[:,i], mat[:,i+1])
    return score

In [5]:
# gold key score
print(colscore(plainmat))
print(colscore(perm_plainmat))

-991.3825477818259
-1311.2913126693559


In [6]:
# construct score matrix for the permuted character matrix

A = np.zeros((18, 18))
for i in range(18):
    for j in range(18):
        if i == 17 or j == 17:
            A[i,j] = 0
        else:
            A[i,j] = col2col(perm_plainmat[:,i], perm_plainmat[:,j])
        
np.savetxt('../java/distance',A[:17,:17],delimiter=' ',newline='\n')
n = 18

In [7]:
print(A)

[[ -83.43130355  -74.39855826  -74.65329408  -85.525534    -80.23126393
   -75.40640933  -85.43333816  -77.28422436  -70.72758208  -76.69292751
   -80.22987545  -86.24933787  -83.00904991  -81.70629723  -82.4490424
   -90.37613178  -77.60148705    0.        ]
 [ -78.67537404  -96.77939275  -92.62516092  -85.15695241  -90.22019749
   -78.08644456  -76.89270968  -77.43592763  -80.45634605  -79.86814205
   -83.4495581   -79.21911996  -69.14579207  -76.66463537  -98.85191663
   -58.63878414  -77.3844895     0.        ]
 [ -72.9176859   -72.25054455  -90.88885453  -76.65051256  -84.15738131
   -80.63680378  -84.80348356  -80.75335664  -88.04474809  -84.51278223
   -72.92973144  -83.20847167  -79.47159828  -83.20856277  -83.35283867
   -74.54372219  -81.00672702    0.        ]
 [ -75.1700495   -81.33001169  -86.01410096 -105.55895145  -83.31328668
   -81.61508458 -106.08228409  -81.88433167  -74.74782928  -85.5160752
   -99.53613011  -69.18248731  -88.61931681  -79.18325266  -88.09257254
   

In [8]:
def subtour(B):
    """
    return subtour from a boolean matrix B
    """
    node = 0
    subt = [node]
    while True:
        for j in range(n):
            #print (B[subt[-1], j])
            if B[j,subt[-1]] > 0.99:
                if j not in subt:
                    subt.append(j)
                else:
                    return subt

In [10]:
"""
MTZ subtour elimination constraint
"""

# boolean matrix, indicating the trip
B = cvx.Bool(n,n)

# exemplary matrix
C = np.ones((1,n), dtype=int)

# auxiliary var
# u = cvx.Variable(n, integer=True)
u = cvx.Int(n)

# objective
obj = cvx.Maximize(sum([A[i,:]*B[:,i] for i in range(n)]))


# basic condition
constraints = [(cvx.sum_entries(B, axis=0) == C), (cvx.sum_entries(B, axis=1) == C.transpose())]

# subtour elimination
for i in range(1,n):
    for j in range(1,n):
        if i != j:
            constraints.append(u[i] - u[j] + n*B[i,j] <= n - 1)
            
# condition for u
for i in range(1,n):
    constraints.append(u[i] >= 0)
    constraints.append(u[i] <= n - 1)
    
st = timeit.default_timer()
prob = cvx.Problem(obj, constraints)

# Time performance:

opt = prob.solve(solver=cvx.GLPK_MI)

# Print results
print ("Minimal time: ", opt)
print ("Optimal tour: ", subtour(B.value))
print ("Converge time: ", timeit.default_timer() - st)


Minimal time:  -991.3825477818259
Optimal tour:  [0, 8, 1, 15, 2, 10, 5, 12, 11, 3, 17, 7, 13, 4, 6, 14, 9, 16]
Converge time:  0.3370185359963216


In [ ]:
# re-map and print
tour = subtour(B.value)
dummy_node_idx = tour.index(17)
tour = tour[dummy_node_idx+1:] + tour[:dummy_node_idx]
print(tour)
proposal = perm_plainmat[:,tour]
print(proposal)
colscore(proposal)
B = B.value
print(sum([A[i,:]*B[:,i] for i in range(n)]))
print(B)
print(u.value)

In [ ]:
"""
Lazy subtour elimination
"""

# boolean matrix, indicating the trip
B = cvx.Bool(n,n)

# exemplary matrix
C = np.ones((1,n), dtype=int)

# objective
obj = cvx.Maximize(sum([A[i,:]*B[:,i] for i in range(n)]))

# basic condition
constraints = [(cvx.sum_entries(B, axis=0) == C), (cvx.sum_entries(B, axis=1) == C.transpose())]

# preliminary solution, which might involve subtours
prob = cvx.Problem(obj, constraints)
st = timeit.default_timer()
opt = prob.solve()

# while True:
#     subt = subtour(B.value)
#     if len(subt) == n:
#         print ("Minimal time: ", opt)
#         print ("Optimal tour: ", subt)
#         print ("Converge time: ", timeit.default_timer() - st)
#         break
#     else:
#         print ("Try: ", subt)
#         nots = [j for j in range(n) if j not in subt]
#         constraints.append(sum(B[i,j] for i in subt for j in nots) >= 1)
#         prob = cvx.Problem(obj, constraints)
#         opt = prob.solve(solver=cvx.GLPK_MI)

In [ ]:
B.value